In [ ]:
import warnings
from causalnex.structure import StructureModel

warnings.filterwarnings("ignore")  # silence warnings

sm = StructureModel()

In [ ]:
sm.add_edges_from([
    ('health', 'absences'),
    ('health', 'G1')
])

In [ ]:
from IPython.display import Image
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
Image(viz.draw(format='png'))

In [ ]:
import pandas as pd

data = pd.read_csv('student-por.csv', delimiter=';')
data.head(5)

In [ ]:
drop_col = ['school','sex','age','Mjob', 'Fjob','reason','guardian']
data = data.drop(columns=drop_col)
data.head(5)

In [ ]:
import numpy as np

struct_data = data.copy()
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)

print(non_numeric_columns)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

struct_data.head(5)

In [ ]:
# struct_data.to_json("../public/student_data.json", orient="records")

In [ ]:
from causalnex.structure.notears import from_pandas

sm = from_pandas(struct_data)

In [ ]:
viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
    prog='fdp',
)
Image(viz.draw(format='png'))

In [ ]:
sm.remove_edges_below_threshold(0.8)
viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
Image(viz.draw(format='png'))

In [ ]:
sm = from_pandas(struct_data, tabu_edges=[("higher", "Medu")], w_threshold=0.8)

In [ ]:
from networkx.readwrite import json_graph
import networkx as nx

pos=nx.spring_layout(sm)

newPos = {}

for p in pos:
    value = pos[p]
    newPos[p] = {"x": value[0], "y": value[1]}

nx.set_node_attributes(sm,newPos)
data = json_graph.node_link_data(sm, {"link": "links", "source": "source", "target": "target", "name": "name"})

nodelink = {}
nodelink["nodes"] = data["nodes"]
nodelink["links"] = data["links"]

In [ ]:
import json

with open("../public/student_nodelink.json", 'w') as fp:
    json.dump(nodelink, fp)